In [1]:
import dask.dataframe as dd
import dask_geopandas as dg
import geopandas as gpd
import numpy as np
import pandas as pd

from segment_speed_utils.project_vars import (SEGMENT_GCS, 
                                              CONFIG_PATH, 
                                              PROJECT_CRS
                                             )
from segment_speed_utils import (helpers, wrangle_shapes, 
                                 segment_calcs)
from shared_utils import rt_dates
from shared_utils.geography_utils import WGS84

import test_split

analysis_date = rt_dates.DATES["jul2023"]

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
from importlib import reload

dict_inputs = helpers.get_parameters(CONFIG_PATH, "stop_segments")

In [3]:
USABLE_VP = dict_inputs["stage1"]
INPUT_FILE = dict_inputs["stage3"]
SEGMENT_FILE = dict_inputs["segments_file"]
SEGMENT_IDENTIFIER_COLS = dict_inputs["segment_identifier_cols"]
GROUPING_COL = dict_inputs["grouping_col"]
TIMESTAMP_COL = dict_inputs["timestamp_col"]

In [ ]:
usable_vp = dd.read_parquet(
    f"{SEGMENT_GCS}{USABLE_VP}_{analysis_date}",
    columns = ["trip_instance_key", "vp_idx", TIMESTAMP_COL, "x", "y"]
)
vp_idx_bounds = test_split.get_usable_vp_bounds_by_trip(usable_vp)

In [4]:
from dask import delayed, compute

# Start from pared down vp
df = delayed(pd.read_parquet)(
    f"{SEGMENT_GCS}vp_pare_down/{INPUT_FILE}_all_{analysis_date}",
    columns = SEGMENT_IDENTIFIER_COLS + ["trip_instance_key", "vp_idx"]
)

In [5]:
df = pd.read_parquet(
    f"{SEGMENT_GCS}vp_pare_down/{INPUT_FILE}_all_{analysis_date}",
    #columns = SEGMENT_IDENTIFIER_COLS + ["trip_instance_key", "vp_idx"]
)

In [10]:
def make_wide(
    df: pd.DataFrame, 
    segment_identifier_cols: list,
    timestamp_col: str
) -> pd.DataFrame:
    
    group_cols = ["trip_instance_key"] + segment_identifier_cols
    
    # Sort and make sure we get a pairing of point 1 and 2 within segment
    df = df.sort_values(group_cols + ["vp_idx"]).reset_index(drop=True)
    
    df = dd.from_pandas(df, npartitions=50, sort=True)

    df = df.assign(
        obs = df.groupby(group_cols, 
                         observed=True, group_keys=False
                        ).cumcount() + 1
    )
    
    gdf = dg.from_dask_dataframe(
        df, 
        geometry=dg.points_from_xy(df, "x", "y")
    ).set_crs(WGS84).to_crs(PROJECT_CRS).drop(columns = ["x", "y"])
    
    segments = helpers.import_segments(
        SEGMENT_GCS,
        f"{SEGMENT_FILE}_{analysis_date}", 
        columns = segment_identifier_cols + ["geometry"]
    ).dropna(subset="geometry").reset_index(drop=True)
    
    gddf = dd.merge(
        gdf,
        segments,
        on = segment_identifier_cols,
        how = "inner"
    ).rename(columns = {
        "geometry_x": "vp_geometry",
        "geometry_y": "segment_geometry"
    }).set_geometry("vp_geometry")
    
    shape_meters_series = gddf.map_partitions(
        wrangle_shapes.project_point_geom_onto_linestring,
        "segment_geometry",
        "vp_geometry",
        meta = ("shape_meters", "float")
    )
    
    gddf["shape_meters"] = shape_meters_series
    
    ddf = gddf.drop(columns = ["vp_geometry", "segment_geometry"]).reset_index(drop=True)
    '''
    ddf = segment_calcs.convert_timestamp_to_seconds(
        ddf, [timestamp_col])
    
    # Must do point 1 against point 2
    # Some segments only have 1 point, have no second point to find
    point1 = ddf[ddf.obs==1].reset_index(drop=True)
    point2 = ddf[ddf.obs==2].reset_index(drop=True)
    
    ddf_wide = dd.merge(
        point1,
        point2,
        on = group_cols,
        how = "left"
    ).reset_index(drop=True)
    '''
    return ddf

In [11]:
results = make_wide(
    df, 
    SEGMENT_IDENTIFIER_COLS, 
    TIMESTAMP_COL
).compute()

/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line,

In [12]:
results

,vp_idx,trip_instance_key,location_timestamp_local,shape_array_key,stop_sequence,obs,shape_meters
0,3140711,0000703b8b86da99dbf637e746c45236,2023-07-12 18:14:58,15b81a6fa853534940f7f1c8f7c3a3ba,1,1,0.000000
1,3140712,0000703b8b86da99dbf637e746c45236,2023-07-12 18:15:13,15b81a6fa853534940f7f1c8f7c3a3ba,1,2,0.000000
2,3140711,0000703b8b86da99dbf637e746c45236,2023-07-12 18:14:58,15b81a6fa853534940f7f1c8f7c3a3ba,2,1,13.352249
3,3140719,0000703b8b86da99dbf637e746c45236,2023-07-12 18:17:31,15b81a6fa853534940f7f1c8f7c3a3ba,2,2,1114.127066
4,3140717,0000703b8b86da99dbf637e746c45236,2023-07-12 18:17:00,15b81a6fa853534940f7f1c8f7c3a3ba,3,1,7.565720
...,...,...,...,...,...,...,...
99194,6808154,ffffe3ff18b68f1b90b97f583d4601ef,2023-07-12 19:06:37,f8348bfb848a94699bf0a8d3c2be02c9,20,2,1042.154977
99195,6808154,ffffe3ff18b68f1b90b97f583d4601ef,2023-07-12 19:06:37,f8348bfb848a94699bf0a8d3c2be02c9,21,1,15.118653
99196,6808156,ffffe3ff18b68f1b90b97f583d4601ef,2023-07-12 19:07:17,f8348bfb848a94699bf0a8d3c2be02c9,21,2,393.326336
99197,6808156,ffffe3ff18b68f1b90b97f583d4601ef,2023-07-12 19:07:17,f8348bfb848a94699bf0a8d3c2be02c9,22,1,0.000000


In [ ]:
df2 = delayed(test_split.get_prior_position_on_segment)(
    df, 
    SEGMENT_IDENTIFIER_COLS,
    TIMESTAMP_COL
)

In [ ]:
df3 = delayed(dd.merge)(
    df2,
    vp_idx_bounds,
    on = "trip_instance_key",
    how = "inner"
)

df3 = df3.assign(
    prior_vp_idx = df3.apply(
        lambda x: 
        x.vp_idx + 1 if (x.prior_vp_idx < x.min_vp_idx) and 
        (x.vp_idx + 1 <= x.max_vp_idx)
        else x.prior_vp_idx, 
        axis=1)
).drop(columns = ["trip_instance_key", "min_vp_idx", "max_vp_idx"])
    

In [ ]:
df3 = compute(df3)[0]

In [ ]:
def attach_vp_timestamp_location(
    df: pd.DataFrame,
    usable_vp: dd.DataFrame,
    timestamp_col: str
) -> gpd.GeoDataFrame:
    """
    """
    
    # Turn the vp_idx we need into gdf
    vp_to_keep = np.union1d(df.vp_idx, df.prior_vp_idx).tolist()
    usable_vp2 = usable_vp[usable_vp.vp_idx.isin(vp_to_keep)]
    
    # Merge in the timestamp and x, y coords 
    usable_gdf = gpd.GeoDataFrame(
        usable_vp2,
        geometry = gpd.points_from_xy(usable_vp2.x, usable_vp2.y),
        crs = WGS84
    ).to_crs(PROJECT_CRS).drop(columns = ["x", "y"])
    
    
    df_with_xy = pd.merge(
        usable_gdf,
        df,
        on = "vp_idx",
        how = "inner"
    ).rename(columns = {"geometry": "vp_geometry"})
    
    # Merge again to get timestamp and x, y coords of previous point
    usable_gdf2 = usable_gdf.rename(
        columns = {
            "vp_idx": "prior_vp_idx",
            timestamp_col: f"prior_{timestamp_col}",
            "geometry": "prior_vp_geometry"
        }
    ).drop(columns = "trip_instance_key")
    
    df_with_prior_xy = dd.merge(
        df_with_xy,
        usable_gdf2,
        on = "prior_vp_idx",
        how = "inner"
    )
    
    return df_with_prior_xy

In [ ]:
from shared_utils.geography_utils import WGS84

gdf = delayed(attach_vp_timestamp_location)(
    df3,
    usable_vp,
    TIMESTAMP_COL
).persist()

In [ ]:
part2 = gdf[gdf.n_vp_seg==2]

part2_keep = (part2.groupby(["trip_instance_key"] + SEGMENT_IDENTIFIER_COLS,
                            observed=True, group_keys=False)
              .vp_idx
              .max()
              .reset_index()
             )

In [ ]:
part2_pared = delayed(dd.merge)(
    part2,
    part2_keep,
    on = ["trip_instance_key", "vp_idx"] + SEGMENT_IDENTIFIER_COLS, 
    how = "inner"
)


In [ ]:
part2_gdf = delayed(test_split.merge_in_segments)(
    part2_pared,
    SEGMENT_IDENTIFIER_COLS,
    GROUPING_COL,
    n_vp_seg_value=2
)

In [ ]:
p2_gdf = compute(part2_gdf)[0]

In [ ]:
gddf = dg.from_geopandas(p2_gdf, npartitions=100)

In [ ]:
shape_meters_series = gddf.map_partitions(
    wrangle_shapes.project_point_geom_onto_linestring,
    "geometry",
    "vp_geometry",
    meta = ("shape_meters", "float")
)

prior_shape_meters_series = gddf.map_partitions(
    wrangle_shapes.project_point_geom_onto_linestring,
    "geometry",
    "prior_vp_geometry",
    meta = ("prior_shape_meters", "float")
)

In [ ]:
gddf["difference_shape_meters"] = abs(
        shape_meters_series - prior_shape_meters_series)
gddf["straight_distance"] = gddf.vp_geometry.distance(gddf.prior_vp_geometry)

In [ ]:
scaling_factor = 1.75
min_meters_elapsed = 100

# It's not only that difference_shape_meters==0 is wrong,
# Separate out these and try again with full shape 
gddf = gddf.assign(
    meters_elapsed = gddf.apply(
        lambda x: x.straight_distance if (
            x.difference_shape_meters <= min_meters_elapsed or 
            x.difference_shape_meters >= x.straight_distance*scaling_factor
        ) else x.difference_shape_meters, 
        axis=1, 
        meta = ("meters_elapsed", "float")
    ),
)

gddf = segment_calcs.convert_timestamp_to_seconds(
    gddf, [TIMESTAMP_COL, f"prior_{TIMESTAMP_COL}"])

gddf = gddf.assign(
    sec_elapsed = (gddf[f"{TIMESTAMP_COL}_sec"] - 
                      gddf[f"prior_{TIMESTAMP_COL}_sec"]).abs()
)

In [ ]:
ddf = gddf.drop(columns = ["prior_vp_geometry", 
                           "vp_geometry", "geometry"])

In [ ]:
ddf = ddf.repartition(npartitions=2)

In [ ]:
results = ddf.compute()

In [ ]:
from shared_utils.rt_utils import MPH_PER_MPS

results = results.assign(
    speed_mph = (results.meters_elapsed.divide(results.sec_elapsed) * 
                 MPH_PER_MPS)
)

In [ ]:
results[results.speed_mph < 5].shape

In [ ]:
ok_results = results[results.speed_mph >= 5]

In [ ]:
bad_results = results[(results.speed_mph >= 0) & (results.speed_mph <5)]

In [ ]:
len(ok_results), len(bad_results)

In [ ]:
bad_results

In [ ]:
part1 = gdf[gdf.n_vp_seg==1]

p1 = compute(part1)[0]

p1.to_parquet("one_vp_in_seg.parquet")

In [ ]:
bad_shapes = bad_results.shape_array_key.unique().tolist()

In [ ]:
ok_results.to_parquet("ok_results.parquet")
bad_results.to_parquet("bad_results.parquet")

For speeds that are unusually low, it is now confirmed it's not just `loop_or_inlining`. It happens on `loop_or_inlining==0` too, so it's good to have moved the sjoin postprocessing to all shapes.

In [ ]:
segments = gpd.read_parquet(
    f"{SEGMENT_GCS}stop_segments_{analysis_date}.parquet",
    filters = [[(GROUPING_COL, "in", bad_shapes)]],
    columns = SEGMENT_IDENTIFIER_COLS + ["loop_or_inlining", "geometry"]
)

In [ ]:
segments.loop_or_inlining.value_counts()